In [13]:
import pandas as pd
import json
from haversine import haversine
import datetime
import os
import numpy as np
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))
os.getcwd()

'/home/jian/Projects/Perry_Ellis'

In [14]:
store_list=pd.read_excel("/home/jian/Projects/Perry_Ellis/Perry Ellis - Major Department Stores.xlsx",dtype=str)
store_list['latitude']=store_list['latitude'].astype(float)
store_list['longitude']=store_list['longitude'].astype(float)
store_list['zip_code']=store_list['zip_code'].apply(lambda x: x.zfill(5))
store_list['Store_Comb']=store_list['list_name']+"|"+store_list['address']+"|"+store_list['city']+"|"+store_list['state']+"|"+store_list['zip_code']


In [15]:
i_counter=0
output_long=pd.DataFrame()
output_wide=store_list.copy()
output_wide['zip_list']=np.nan
for i in range(len(store_list)):
    store_name=store_list['Store_Comb'][i]
    store_center=[store_list['latitude'][i],store_list['longitude'][i]]
    zip_list=[store_list['zip_code'][i]]
    df_store=pd.DataFrame({"Store_Comb":store_name,"zip_cd":zip_list,"dist_miles":"store_zip"},index=[i_counter])
    
    i_counter+=1
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            zip_list=zip_list+[zip_cd]
            df=pd.DataFrame({"Store_Comb":store_name,"zip_cd":zip_cd,"dist_miles":dist},index=[i_counter])
            df_store=df_store.append(df)
    zip_list=set(zip_list)
    output_long=output_long.append(df_store)
    output_wide['zip_list'][i]=[zip_list]
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
output_long_unique=output_long.sort_values(['zip_cd','dist_miles']).drop_duplicates(['zip_cd'])

In [20]:
writer=pd.ExcelWriter("/home/jian/Projects/Perry_Ellis/Perry Ellis - Major Department Stores_With_Zips_in_10_miles_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_long_unique.to_excel(writer,"unique_zips",index=False)
output_wide.to_excel(writer,"store_list_with_zips",index=False)
output_long.to_excel(writer,"zips_by_store",index=False)
writer.save()

In [18]:
output_long.head(2)

,Store_Comb,dist_miles,zip_cd
0,Macy's|54 East Main Street|Walla Walla|WA|99362,store_zip,99362
1,Macy's|54 East Main Street|Walla Walla|WA|99362,2.83511,99324
